In [1]:
from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml import command
from azure.ai.ml.entities import Environment
from azure.ai.ml.constants import AssetTypes

In [2]:
aml_sub_id = ''
aml_rg_name = ''
aml_ws_name = ''

In [3]:
experiment_env_vars = {
    'LLM_NAME': 'Codestral-2501',
    'CODE_GEN_ITERATIONS': '2',
    'LLM_ENDPOINT_PATH': '',
    'LLM_CREDENTIAL': '',
    'LLM_TEMP': 0.5,
    'LLM_TOP_P': 0.1
}

In [ ]:
credential = DefaultAzureCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id=aml_sub_id,
    resource_group_name=aml_rg_name,
    workspace_name=aml_ws_name,
)

In [ ]:
env_name = 'code-gen experiments'

job_env = Environment(
    name=env_name,
    description='Custom AML environment for code-gen experiments',
    tags={
        'python': '3.10'
    },
    conda_file='./experiment_conda.yaml',
    image='mcr.microsoft.com/azureml/curated/python-sdk-v2:25',
)

job_env = ml_client.environments.create_or_update(job_env)

In [ ]:
job = command(
    inputs= {
        'test_cases_path': Input(path='./test_cases', type=AssetTypes.URI_FOLDER)
    },
    code='./experiment/',
    command='python main.py --test_cases_path ${{inputs.test_cases_path}}',
    environment=job_env,
    environment_variables=experiment_env_vars,
    experiment_name='code-gen with codebleu'
)

In [ ]:
ml_client.create_or_update(job)